# KDD Cup MOOC Dataset

- dataset preparation and exploration
- dataset preparation step is similar to XuetangX dataset
- this dataset doesn't provide user demographics (age, sex, education, etc.)

In [1]:
import os
import pandas as pd
import numpy as np
import pickle as pkl
import math
from sklearn.preprocessing import StandardScaler

In [2]:
data_path = 'data/kddcup15'

# load training log
train_df = pd.read_csv(os.path.join(data_path, 'train/log_train.csv'))

In [3]:
pd.options.display.max_columns = 30
pd.options.display.max_rows = 20

In [3]:
train_df.head()

,enrollment_id,time,source,event,object
0,1,2014-06-14T09:38:29,server,navigate,Oj6eQgzrdqBMlaCtaq1IkY6zruSrb71b
1,1,2014-06-14T09:38:39,server,access,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl
2,1,2014-06-14T09:38:39,server,access,qxvBNYTfiRkNcCvM0hcGwG6hvHdQwnd4
3,1,2014-06-14T09:38:48,server,access,2cmZrZW2h6Il91itO3e89FGcABLWhf3W
4,1,2014-06-14T09:41:49,browser,problem,RMtgC2bTAqEeftenUUyia504wsyzeZWf


In [4]:
train_df.tail()

,enrollment_id,time,source,event,object
8157272,200901,2014-07-24T14:11:31,browser,page_close,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl
8157273,200901,2014-07-24T14:11:32,browser,video,HdMvr3A6vQzym6Xl0tOXpNbfHOyohlKE
8157274,200901,2014-07-24T14:11:41,browser,page_close,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl
8157275,200904,2014-07-24T15:07:50,server,navigate,9Mur5ciTV9IBFfcPaz5c3nC1lrZaxBvG
8157276,200905,2014-07-24T15:41:51,server,navigate,9Mur5ciTV9IBFfcPaz5c3nC1lrZaxBvG


In [5]:
# read the ground truch for training data
train_truth_df = pd.read_csv(os.path.join(data_path, 'train/truth_train.csv'), header=None, index_col=0)

In [6]:
train_truth_df.head()

,1
0,
1,0
3,0
4,0
5,0
6,0


In [7]:
train_truth_df.columns = ['truth']

In [8]:
train_truth_df.index.name = 'enrollment_id'

In [9]:
train_truth_df.tail()

,truth
enrollment_id,
200898,1
200900,1
200901,1
200904,1
200905,1


In [10]:
# load test logs
test_df = pd.read_csv(os.path.join(data_path, 'test/log_test.csv'))
test_truth_df = pd.read_csv(os.path.join(data_path, 'test/truth_test.csv'), header=None, index_col=0)

In [11]:
test_truth_df.columns = ['truth']
test_truth_df.index.name = 'enrollment_id'

# cobmine train and test truth
all_truth_df = pd.concat([train_truth_df, test_truth_df])

# combine train and test logs
all_log_df = pd.concat([train_df, test_df])

In [12]:
all_log_df.head()

,enrollment_id,time,source,event,object
0,1,2014-06-14T09:38:29,server,navigate,Oj6eQgzrdqBMlaCtaq1IkY6zruSrb71b
1,1,2014-06-14T09:38:39,server,access,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl
2,1,2014-06-14T09:38:39,server,access,qxvBNYTfiRkNcCvM0hcGwG6hvHdQwnd4
3,1,2014-06-14T09:38:48,server,access,2cmZrZW2h6Il91itO3e89FGcABLWhf3W
4,1,2014-06-14T09:41:49,browser,problem,RMtgC2bTAqEeftenUUyia504wsyzeZWf


In [13]:
all_log_df.tail(10)

,enrollment_id,time,source,event,object
5387837,200894,2014-07-24T10:49:10,server,access,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl
5387838,200894,2014-07-24T10:49:10,server,access,jcmKbpHQYLyzZZ34cJgLu7F4pgiCsoXV
5387839,200894,2014-07-24T10:54:17,browser,page_close,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl
5387840,200894,2014-07-24T10:54:17,browser,video,HdMvr3A6vQzym6Xl0tOXpNbfHOyohlKE
5387841,200894,2014-07-24T10:57:03,server,navigate,9Mur5ciTV9IBFfcPaz5c3nC1lrZaxBvG
5387842,200899,2014-07-24T13:37:53,server,navigate,9Mur5ciTV9IBFfcPaz5c3nC1lrZaxBvG
5387843,200902,2014-07-24T14:22:33,server,navigate,9Mur5ciTV9IBFfcPaz5c3nC1lrZaxBvG
5387844,200903,2014-07-24T14:24:29,server,navigate,9Mur5ciTV9IBFfcPaz5c3nC1lrZaxBvG
5387845,200903,2014-07-24T14:24:41,server,navigate,Oj6eQgzrdqBMlaCtaq1IkY6zruSrb71b
5387846,200903,2014-07-24T14:25:26,server,navigate,LMYZjRiU5C2N9ih1oYVNmOe5jFu2XLwv


In [14]:
# remove duplicate enroll_ids
train_enroll_ids = list(set(list(train_df['enrollment_id'])))
test_enroll_ids = list(set(list(test_df['enrollment_id'])))

In [15]:
# let's check total # of records on train and test datasets
print(len(train_enroll_ids))
print(len(test_enroll_ids))

120542
80362


In [16]:
# count all the actions for each user
user_action_count_df = all_log_df.groupby('enrollment_id').count()[['event']]

In [17]:
user_action_count_df.head(10)

,event
enrollment_id,
1,314
2,875
3,288
4,99
5,633
6,23
7,479
8,353
9,97


In [18]:
# give columns names
user_action_count_df.columns = ['action_count']

In [19]:
user_action_count_df.head(10)

,action_count
enrollment_id,
1,314
2,875
3,288
4,99
5,633
6,23
7,479
8,353
9,97


In [20]:
user_events_df = all_log_df[['event']].drop_duplicates()

In [21]:
user_events_df

,event
0,navigate
1,access
4,problem
12,page_close
69,video
484,discussion
2655,wiki


In [22]:
user_events_list = user_events_df['event'].values.tolist()

In [23]:
user_events_list

['navigate', 'access', 'problem', 'page_close', 'video', 'discussion', 'wiki']

In [24]:
sources_df = all_log_df[['source']].drop_duplicates()

In [25]:
sources_df

,source
0,server
4,browser


In [26]:
sources_list = sources_df['source'].values.tolist()

In [27]:
sources_list

['server', 'browser']

In [28]:
for source in sources_list:
    for action in user_events_list:
        action_label = f'{source}_{action}_count'
        action_ = ((all_log_df['source'] == source) & (all_log_df['event'] == action)).astype(int)
        #print(action_label)
        #print(action_)
        all_log_df[action_label] = action_
        action_count = all_log_df.groupby(['enrollment_id']).sum()[[action_label]]
        user_action_count_df = pd.merge(user_action_count_df, action_count, left_index=True, right_index=True)

In [29]:
user_action_count_df.head(10)

,action_count,server_navigate_count,server_access_count,server_problem_count,server_page_close_count,server_video_count,server_discussion_count,server_wiki_count,browser_navigate_count,browser_access_count,browser_problem_count,browser_page_close_count,browser_video_count,browser_discussion_count,browser_wiki_count
enrollment_id,,,,,,,,,,,,,,,
1,314,25,86,8,0,0,0,0,0,21,79,66,29,0,0
2,875,87,223,4,0,0,324,12,0,16,16,143,50,0,0
3,288,14,45,3,0,0,26,0,0,34,135,22,9,0,0
4,99,15,64,1,0,0,0,0,0,0,5,10,4,0,0
5,633,30,106,32,0,0,34,0,0,120,138,87,86,0,0
6,23,5,12,0,0,0,0,0,0,0,2,2,2,0,0
7,479,20,81,20,0,0,33,0,0,122,74,60,69,0,0
8,353,20,108,7,0,0,7,1,0,19,43,90,58,0,0
9,97,12,65,3,0,0,0,0,0,6,3,6,2,0,0


In [30]:
user_action_count_df.describe()

,action_count,server_navigate_count,server_access_count,server_problem_count,server_page_close_count,server_video_count,server_discussion_count,server_wiki_count,browser_navigate_count,browser_access_count,browser_problem_count,browser_page_close_count,browser_video_count,browser_discussion_count,browser_wiki_count
count,200904.000000,200904.000000,200904.000000,200904.000000,200904.0,200904.0,200904.000000,200904.000000,200904.0,200904.000000,200904.000000,200904.000000,200904.000000,200904.0,200904.0
mean,67.420878,8.359679,19.618539,1.384213,0.0,0.0,5.357395,0.762100,0.0,6.096335,9.018432,10.258701,6.565484,0.0,0.0
std,139.918512,12.904720,38.431194,5.138802,0.0,0.0,35.715282,4.400607,0.0,21.728713,29.694653,20.919278,14.581310,0.0,0.0
min,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
25%,5.000000,2.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
50%,17.000000,4.000000,5.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,2.000000,1.000000,0.0,0.0
75%,67.000000,10.000000,21.000000,1.000000,0.0,0.0,2.000000,1.000000,0.0,3.000000,5.000000,10.000000,6.000000,0.0,0.0
max,7697.000000,649.000000,3641.000000,483.000000,0.0,0.0,5321.000000,1041.000000,0.0,1122.000000,877.000000,694.000000,536.000000,0.0,0.0


In [31]:
 user_action_count_df = pd.merge(user_action_count_df, all_truth_df, left_index=True, right_index=True)

In [32]:
user_action_count_df.head(10)

,action_count,server_navigate_count,server_access_count,server_problem_count,server_page_close_count,server_video_count,server_discussion_count,server_wiki_count,browser_navigate_count,browser_access_count,browser_problem_count,browser_page_close_count,browser_video_count,browser_discussion_count,browser_wiki_count,truth
enrollment_id,,,,,,,,,,,,,,,,
1,314,25,86,8,0,0,0,0,0,21,79,66,29,0,0,0
2,875,87,223,4,0,0,324,12,0,16,16,143,50,0,0,0
3,288,14,45,3,0,0,26,0,0,34,135,22,9,0,0,0
4,99,15,64,1,0,0,0,0,0,0,5,10,4,0,0,0
5,633,30,106,32,0,0,34,0,0,120,138,87,86,0,0,0
6,23,5,12,0,0,0,0,0,0,0,2,2,2,0,0,0
7,479,20,81,20,0,0,33,0,0,122,74,60,69,0,0,1
8,353,20,108,7,0,0,7,1,0,19,43,90,58,0,0,0
9,97,12,65,3,0,0,0,0,0,6,3,6,2,0,0,1


In [33]:
user_action_count_df.shape

(200904, 16)

In [34]:
numeric_features = [c for c in user_action_count_df.columns if 'count' in c or 'time' in c or 'num' in c]

In [35]:
numeric_features

['action_count',
 'server_navigate_count',
 'server_access_count',
 'server_problem_count',
 'server_page_close_count',
 'server_video_count',
 'server_discussion_count',
 'server_wiki_count',
 'browser_navigate_count',
 'browser_access_count',
 'browser_problem_count',
 'browser_page_close_count',
 'browser_video_count',
 'browser_discussion_count',
 'browser_wiki_count']

In [36]:
scaler = StandardScaler()
scaled_df = scaler.fit_transform(user_action_count_df[numeric_features])

In [37]:
for i, n_f in enumerate(numeric_features):
    print(i, n_f)
    user_action_count_df[n_f] = scaled_df[:,i]

0 action_count
1 server_navigate_count
2 server_access_count
3 server_problem_count
4 server_page_close_count
5 server_video_count
6 server_discussion_count
7 server_wiki_count
8 browser_navigate_count
9 browser_access_count
10 browser_problem_count
11 browser_page_close_count
12 browser_video_count
13 browser_discussion_count
14 browser_wiki_count


In [38]:
user_action_count_df.head(10)

,action_count,server_navigate_count,server_access_count,server_problem_count,server_page_close_count,server_video_count,server_discussion_count,server_wiki_count,browser_navigate_count,browser_access_count,browser_problem_count,browser_page_close_count,browser_video_count,browser_discussion_count,browser_wiki_count,truth
enrollment_id,,,,,,,,,,,,,,,,
1,1.762310,1.289479,1.727285,1.287421,0.0,0.0,-0.150003,-0.173181,0.0,0.685899,2.356712,2.664597,1.538584,0.0,0.0,0
2,5.771796,6.093934,5.292106,0.509028,0.0,0.0,8.921766,2.553722,0.0,0.455788,0.235113,6.345421,2.978788,0.0,0.0,0
3,1.576487,0.437075,0.660441,0.314429,0.0,0.0,0.577978,-0.173181,0.0,1.284187,4.242578,0.561268,0.166962,0.0,0.0,0
4,0.225697,0.514567,1.154832,-0.074767,0.0,0.0,-0.150003,-0.173181,0.0,-0.280567,-0.135325,-0.012367,-0.175944,0.0,0.0,0
5,4.042214,1.676935,2.247697,5.957782,0.0,0.0,0.801973,-0.173181,0.0,5.242094,4.343606,3.668458,5.447708,0.0,0.0,0
6,-0.317478,-0.260346,-0.198239,-0.269366,0.0,0.0,-0.150003,-0.173181,0.0,-0.280567,-0.236354,-0.394790,-0.313106,0.0,0.0,0
7,2.941570,0.902023,1.597182,3.622602,0.0,0.0,0.773973,-0.173181,0.0,5.334138,2.188331,2.377779,4.281829,0.0,0.0,1
8,2.041044,0.902023,2.299738,1.092823,0.0,0.0,0.045992,0.054061,0.0,0.593855,1.144369,3.811867,3.527437,0.0,0.0,0
9,0.211403,0.282093,1.180853,0.314429,0.0,0.0,-0.150003,-0.173181,0.0,-0.004434,-0.202678,-0.203578,-0.313106,0.0,0.0,1


In [43]:
user_action_count_df.loc[train_enroll_ids].to_csv(os.path.join(data_path, 'kdd_train_normalized_features.csv'))
user_action_count_df.loc[test_enroll_ids].to_csv(os.path.join(data_path, 'kdd_test_normalized_features.csv'))

In [40]:
# save single file with all features
user_action_count_df.to_csv(os.path.join(data_path, 'kdd_all_normalized_features.csv'))